In [15]:
import logging
import os
import openai
import itertools


from actionweaver.llms.openai.tools.chat import OpenAIChatCompletion
from actionweaver.llms.openai.tools.tokens import TokenUsageTracker
from actionweaver import action

openai.api_key = os.getenv("OPENAI_API_KEY")


 

def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')


In [16]:
logging.basicConfig(
    filename='parallel_tools.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [17]:

@action(name="GetWeather")
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


In [18]:
chat = OpenAIChatCompletion("gpt-3.5-turbo-1106", logger=logger)

In [19]:
print_output(chat.create([{"role": "user", "content": "what time is it now and What's the weather like in San Francisco, Tokyo, and Paris?"}], actions = [get_current_time, get_current_weather], stream=False))



The current time is 19:53 and the weather is as follows:
- San Francisco: 72°F
- Tokyo: 10°C
- Paris: 22°C


In [20]:
print_output(get_current_time.invoke(chat, [{"role": "user", "content": "what time is it ?"}], stream=True))


The current time is 7:53 PM.

In [21]:
from pydantic import BaseModel
from actionweaver.actions.factories.pydantic_model_to_action import action_from_model

class User(BaseModel):
    name: str
    age: int

action_from_model(User, stop=True).invoke(chat, [{"role": "user", "content": "Tom is 31 years old"}])

KeyboardInterrupt: 